<a href="https://colab.research.google.com/github/waqasm86/Kaggle-Dropbox-HuggingFace/blob/main/p3_colab_dropbox_huggingface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U huggingface_hub dropbox requests > /dev/null

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.3, but you have requests 2.32.5 which is incompatible.


In [2]:
!pip -q install -U  tqdm requests transformers accelerate bitsandbytes sentencepiece einops safetensors openai >/dev/null

In [3]:
import torch, os, json, pathlib, gc

In [4]:
import os, json, pathlib, posixpath, re, sys, time
from typing import Optional, Iterable
import dropbox
from dropbox.exceptions import AuthError, ApiError

In [6]:
import os, pathlib
from google.colab import userdata

# ---- Dropbox (preferred: offline/refreshable) ----
os.environ["DBX_APP_KEY"]       = userdata.get('DBX_APP_KEY')
os.environ["DBX_APP_SECRET"]    = userdata.get('DBX_APP_SECRET')
os.environ["DBX_REFRESH_TOKEN"] = userdata.get('DBX_REFRESH_TOKEN')
os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN')

DBX_MODEL_DIR  = "/p1-gemma-3-1b-it"         # your Dropbox folder from Kaggle step
LOCAL_MODEL_DIR = "/content/gemma-3-1b-it"   # Colab temp storage
pathlib.Path(LOCAL_MODEL_DIR).mkdir(parents=True, exist_ok=True)

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="google/gemma-3-1b-it")
messages = [
    {"role": "user", "content": "Who are you?"},
]
pipe(messages)

config.json:   0%|          | 0.00/899 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

Device set to use cpu


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'Hi there! I’m Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m publicly available for anyone to use! \n\nI’m designed to take text and images as input and produce text as output. \n\nHow can I help you today?'}]}]

In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-1b-it")
model = AutoModelForCausalLM.from_pretrained("google/gemma-3-1b-it")
messages = [
    {"role": "user", "content": "Who are you?"},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

outputs = model.generate(**inputs, max_new_tokens=40)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Hi there! I’m Gemma, a large language model created by the Gemma team at Google DeepMind. I’m an open-weights model, which means I’m publicly available for anyone
